In [1]:
import json
import pika
import uuid
import pybars
import requests
import psycopg2

In [2]:
DNS="host='localhost' dbname='ChatbotVB' user='postgres' password='2011080606'"
con=psycopg2.connect(DNS)
cur=con.cursor()

In [3]:
def consulta(query):
    global cur
    cur.execute(query)
    dato=cur.fetchone()
    return dato

In [4]:
def consulta2(query):
    global cur
    cur.execute(query)
    dato=cur.fetchall()
    return dato

In [5]:
def insertar(query):
    global cur
    global con
    cur.execute(query)
    con.commit()

In [6]:
def cerrarConexionDB():
    global cur
    global con
    cur.close()
    con.close()

In [7]:
def mensInf(state,mensaje):
    n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"][0]["content"] = str(consulta("select contenido from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["contentType"]= str(consulta("select contentType from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"][0]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='informativo' where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
    pay=json.dumps(payload)
    decoded["Respuesta"][0]["payload"].append(json.loads(pay))
    return decoded

In [8]:
def mensQr(state,mensaje):
    n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
    opc=(consulta2("select opciones from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    opciones=opc[0].split(",")
        
    decoded["Respuesta"][0]["content"] = str(consulta("select contenido from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["options"] = opciones
    decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["default_id"] = str(consulta("select default_id from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"][0]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='quickReply' where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
    pay=json.dumps(payload)
    decoded["Respuesta"][0]["payload"].append(json.loads(pay))
    return decoded

In [9]:
def mensInp(state,mensaje):
    n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"][0]["content"] = str(consulta("select contenido from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["contentType"]= str(consulta("select contentType from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["validacion"]= str(consulta("select validacion from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["Default_id"] = str(consulta("select default_id from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"][0]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='input' where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
    pay=json.dumps(payload)
    decoded["Respuesta"][0]["payload"].append(json.loads(pay))
    return decoded

In [10]:
def mensInfDin(state,mensaje):
    n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
    
    id_bloque=consulta("select id_block from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+id_bloque+"' and blocktype='informativoDinamico'")
    name=consulta2("select namecredencial from credencialesapis where id_block='"+id_bloque+"' and blocktype='informativoDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+id_bloque+"' and blocktype='informativoDinamico'")
    contenido=consulta("select contenido from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0]
    credenciales={}
    cont=0
    for nm in name:
        credenciales[nm[0]]=cr[cont][0]        
        cont=cont+1
    
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"][0]["entities"])
    output=ingresarVar(contenido, "",mensaje["payload"][0]["entities"])
    content=crearRespuesta(output, respAPI, mensaje["payload"][0]["entities"])
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"][0]["content"] = content   
    decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["contentType"]= str(consulta("select contentType from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"][0]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='informativoDinamico' where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
    pay=json.dumps(payload)
    decoded["Respuesta"][0]["payload"].append(json.loads(pay))
    return decoded

In [11]:
def mensInpDin(state,mensaje):
    n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
    
    id_bloque=consulta("select id_block from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0]
    save_var=consulta("select save_var from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+id_bloque+"' and blocktype='inputDinamico'")
    name=consulta2("select namecredencial from credencialesapis where id_block='"+id_bloque+"' and blocktype='inputDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+id_bloque+"' and blocktype='inputDinamico'")
    contenido=consulta("select contenido from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0]
    credenciales={}
    cont=0
    for nm in name:
        credenciales[nm[0]]=cr[cont][0]        
        cont=cont+1
    
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"][0]["entities"])
    output=ingresarVar(contenido, save_var, mensaje["payload"][0]["entities"])
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"][0]["content"] = output   
    decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["contentType"]= str(consulta("select contentType from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["validacion"]= str(consulta("select validacion from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["Default_id"] = str(consulta("select default_id from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"][0]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='inputDinamico' where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
    pay=json.dumps(payload)
    decoded["Respuesta"][0]["payload"].append(json.loads(pay))
    return decoded

In [12]:
def mensQRDin(state,mensaje):
    n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
    
    id_bloque=consulta("select id_block from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0]
    links=consulta2("select links from linksapis where id_block='"+id_bloque+"' and blocktype='quickReplyDinamico'")
    name=consulta2("select namecredencial from credencialesapis where id_block='"+id_bloque+"' and blocktype='quickReplyDinamico'")
    cr=consulta2("select credencial from credencialesapis where id_block='"+id_bloque+"' and blocktype='quickReplyDinamico'")    
    credenciales={}
    cont=0
    for nm in name:
        credenciales[nm[0]]=cr[cont][0]        
        cont=cont+1
        
    print("Datos bloque QR Dinamico=>")
    print("id_bloque-> ",id_bloque)
    print("links-> ",links)
    print("credenciales-> ",credenciales)
    
    respAPI=consultarAPIs(links, credenciales, mensaje["payload"][0]["entities"])
    options=generarOpc(respAPI)
    data_string = json.dumps(n)
    decoded = json.loads(data_string)
        
    decoded["Respuesta"][0]["content"] = str(consulta("select contenido from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["options"] = options
    decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])        
    decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])    
    decoded["Respuesta"][0]["Default_id"] = str(consulta("select default_id from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    decoded["Respuesta"][0]["save_var"] = str(consulta("select save_var from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")[0])
    
    mensajeAct=str(mensaje["payload"][0]["entities"])
    insertar("update ticket set entities='"+mensajeAct.replace("'", '"')+"', estado_actual='"+state+"', block_type='quickReplyDinamico' where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
    insertar("update bloqueqrdinamico set opciones='"+options+"' where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'")
    payload={"content":"","state":state,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
    pay=json.dumps(payload)
    decoded["Respuesta"][0]["payload"].append(json.loads(pay))
    return decoded

In [13]:
def generarOpc(re):
    key=""
    if(len(re)>1):
        print("Mayor a 1")   
    else:
        for raiz in re:
            key=raiz
        opciones=list(re[key][0].keys())
    
    return opciones

In [14]:
def ingresarVar(content, nameVar, entities):
    prueba1={}
    prueba2={}
    principal=""    
    
    output, var=completarURLs(content, prueba1, prueba2, entities)
    if(nameVar!=""):
        entities[nameVar]=var        
    
    return output

In [15]:
def consultarAPIs(links, tockens, entities):
    print("consultarAPIs-> ")
    print("links-> ", links)
    print("tockens-> ", tockens)
    print("entities-> ", entities)
    re={}
    prueba1={}
    prueba2={}
    if(len(links)>0): 
        for link in links:      
            #print("\n\nDatos enviados de consultarAPIs a completarURLs")
            #print("\n\nURl 1= ",links[link])
            #print("\n\nTockens = ",tockens)
            #print("\n\nConsulta 1= ",re)
            #print("\n\nEtidades 1= ",entities)
            urlF, var=completarURLs(link[0], tockens, re, entities)           
            r = requests.get(urlF)
            #print(urlF)
            re=r.json()  
    return re            

In [16]:
def crearRespuesta(content, re, entities):
    nameVar='vacio'
    cont=0
    cont2=0
    if(len(re)>1):
        source = content
        compiler = pybars.Compiler()
        template = compiler.compile(source)
        output = template(re)    
    else:
        for raiz in re:
            key=raiz
        general=re[key]
        for var in entities:
            nameVar=var  
            cont=cont+1;
        for var in entities:
            cont2=cont2+1;
            if cont2<cont:
                nameVar2=var
        #print("general= ",general)
        print("nameVar= ",nameVar2)
        print("entities[nameVar]= ",entities[nameVar])
        #posicion=buscarDatos(general, nameVar2, entities[nameVar])
        posicion=buscarDatos(general, entities[nameVar2], entities[nameVar])
                
        source = content
        compiler = pybars.Compiler()
        template = compiler.compile(source)
        output = template(re[key][posicion]) 
        
    return output

In [17]:
def completarURLs(url, tockens, consulta, entities):
    pos1=1
    pos2=0
    posAux=0
    cadAux=url
    
    var=" "
    while(pos1>-1):
        if(posAux<len(url)):
            pos1=cadAux.find('{')
            pos2=cadAux.find('}')
            if(url[posAux+pos1+1]=='{' and pos1>-1): 
                cadAux=url[pos2+posAux+2:]
                token=url[posAux+pos1+2:pos2+posAux]
                if token in entities:
                    res=len(entities[token])-len(token)
                    url=url[:posAux+pos1+2]+entities[token]+url[pos2+posAux:]
                    posAux=pos1+posAux+res+2
                else:
                    posAux=pos2+posAux+2
            elif(pos1>-1):                                
                cadAux=url[pos2+posAux+1:]                
                token2=url[pos1+posAux+1:pos2+posAux]
                token=token2
                if token.lower() in tockens:
                    url=url[:pos1+posAux]+tockens[token.lower()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                if token.upper() in tockens:
                    url=url[:pos1+posAux]+tockens[token.upper()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                elif token.lower() in consulta:
                    url=url[:pos1+posAux]+consulta[token.lower()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                elif token.upper() in consulta:
                    url=url[:pos1+posAux]+consulta[token.upper()]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                elif token2 in entities:
                    url=url[:pos1+posAux]+entities[token2]+url[pos2+1+posAux:]
                    cadAux=url[pos1+posAux:]
                    posAux=pos1+posAux
                else:
                    posAux=pos2+posAux+1
        else:
            pos1=-1
    print(url)
    return url, var

In [18]:
def buscarDatos(general, nameVar, dataVar): 
    cont=0    
    if nameVar in 'vacio' or dataVar in 'vacio':
        print("vacio")
    else:
        for dato in general:           
            if(type(dato[nameVar])==int):
                dataVar=int(dataVar)
            if(dato[nameVar]==dataVar):
                break       
            cont+=1
    return cont
        

In [19]:
def envia(ch,props, method, data_string):    
    ch.basic_publish(exchange='',
                 routing_key=props.reply_to,
                 properties=pika.BasicProperties(correlation_id = \
                                                     props.correlation_id),
                 body=str(data_string))
    ch.basic_ack(delivery_tag = method.delivery_tag)

In [20]:
def confirma(ch,props, method, state, mensaje):
    cont=1 
    data_string=' '
    print("Siguiente estado...", state)  
    if((str(consulta("select blocktype from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'"))) not in "None"):
        if(state=="Salida"):
            print("Terminando conversación...")
            cont=-100
        resp=mensInf(state,mensaje)
        data_string = json.dumps(resp)        

    elif((str(consulta("select blocktype from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'"))) not in "None"):
        resp=mensQr(state,mensaje)
        data_string = json.dumps(resp)
    
    elif((str(consulta("select blocktype from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'"))) not in "None"):
        resp=mensInp(state,mensaje)
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'"))) not in "None"):
        resp=mensInfDin(state,mensaje)
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'"))) not in "None"):
        resp=mensInpDin(state,mensaje)
        data_string = json.dumps(resp)
        
    elif((str(consulta("select blocktype from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+state+"'"))) not in "None"):
        resp=mensQRDin(state,mensaje)
        data_string = json.dumps(resp)
    
    
    print(" [x] Enviando... ", str(data_string))
    envia(ch,props, method, data_string)
    #print("llegó esto: ",mensaje["payload"])
    return cont #,mensaje;

In [21]:
connection = pika.BlockingConnection(pika.ConnectionParameters(
        host='localhost'))

channel = connection.channel()

channel.queue_declare(queue='chatbotV12')


def on_request(ch, method, props, body):
    mensaje= json.loads(body)
    print("Mensaje User=> ",mensaje)
    #if(mensaje["payload"][0]["state"]=='none' ):
    print("id_usuario-> ",mensaje["payload"][0]["recipient_id"])
    block_type=(consulta("select block_type from ticket where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'"))    
    if(str(block_type) in "None"):
        n=  {"Respuesta": [{"content":" ","next_id":" ", "blockType":" ", "contentType":" ",
                         "typingTime":" ","payload":[]}]}
        data_string = json.dumps(n)
        decoded = json.loads(data_string)
        print('id_robot: ',mensaje["payload"][0]["robot"])
        bloque_inicial,type_blocki=consulta("select block_ini, type_blocki from robots where id_robot='"+mensaje["payload"][0]["robot"]+"'")
        insertar("insert into ticket (id_robot, id_usuario, estado_actual, block_type ,entities,fecha_mod) values ('"+mensaje["payload"][0]["robot"]+"','"+mensaje["payload"][0]["recipient_id"]+"','"+bloque_inicial+"','"+type_blocki+"','{}','2019/02/06')")
        
        decoded["Respuesta"][0]["content"] = str(consulta("select contenido from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"][0]["next_id"] = str(consulta("select next_id from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"][0]["blockType"]= str(consulta("select blockType from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"][0]["contentType"]= str(consulta("select contentType from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+bloque_inicial+"'")[0])
        decoded["Respuesta"][0]["typingTime"]= str(consulta("select typingTime from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+bloque_inicial+"'")[0])
        
        payload={"content":"","state":bloque_inicial,"recipient_id":mensaje["payload"][0]["recipient_id"],"entities":mensaje["payload"][0]["entities"]}
        pay=json.dumps(payload)
        decoded["Respuesta"][0]["payload"].append(json.loads(pay))
        data_string = json.dumps(decoded)

        print(" [x] Enviando... ", str(data_string))

        envia(ch, props, method,data_string)
    else:
        estado_actual,entitiesConsulta=consulta("select estado_actual,entities from ticket where id_robot='"+mensaje["payload"][0]["robot"]+"' and id_usuario='"+mensaje["payload"][0]["recipient_id"]+"'")
        entitiesDB = json.loads(entitiesConsulta)
        mensaje["payload"][0]["entities"]=entitiesDB
        #if(mensaje["payload"][0]["blocktype"]=="informativo"):
        if(block_type[0]=="informativo"):    
            state=str(consulta("select next_id from bloqueinfo where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            cont=confirma(ch, props, method, state, mensaje)


        #elif(mensaje["payload"][0]["blocktype"]=="quickReply"):
        if(block_type[0]=="quickReply"):
            opciones=consulta("select opciones from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0].split(",")
            cont=0
            for opcion in opciones:
                if opcion in mensaje["payload"][0]["content"]:
                    next_id=consulta("select next_id from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0].split(",")
                    state=str(next_id[cont])
                    break
                cont=cont+1
            else:
                state=str(consulta("select default_id from bloqueqr where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            cont=confirma(ch,props, method, state,mensaje)
            
        #elif(mensaje["payload"][0]["blocktype"]=="input"):
        elif(block_type[0]=="input"):
            dataVar=mensaje["payload"][0]["content"]
            nameVar=str(consulta("select save_var from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            mensaje["payload"][0]["entities"][nameVar]=dataVar
            state=str(consulta("select next_id from bloqueinput where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            cont=confirma(ch,props, method, state,mensaje)
            
        #elif(mensaje["payload"][0]["blocktype"]=="informativoDinamico"):
        elif(block_type[0]=="informativoDinamico"):
            state=str(consulta("select next_id from bloqueinfodinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            cont=confirma(ch,props, method, state,mensaje)
            
        #elif(mensaje["payload"][0]["blocktype"]=="inputDinamico"):
        elif(block_type[0]=="inputDinamico"):
            dataVar=mensaje["payload"][0]["content"]
            save_var=consulta("select save_var from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0]
            if(mensaje["payload"][0]["entities"][save_var]==" "):
                nameVar=save_var
            else:
                nameVar=mensaje["payload"][0]["entities"][save_var]
            mensaje["payload"][0]["entities"][nameVar]=dataVar
            state=str(consulta("select next_id from bloqueinputdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            cont=confirma(ch,props, method, state,mensaje)
            
        #if (mensaje["payload"][0]["blocktype"]=="quickReplyDinamico"):
        elif(block_type[0]=="quickReplyDinamico"):
            opciones=consulta("select opciones from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0].split(",")
            cont=0
            for opcion in opciones:
                if opcion in mensaje["payload"][0]["content"]:
                    dataVar=mensaje["payload"][0]["content"]
                    nameVar=consulta("select save_var from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0]
                    mensaje["payload"][0]["entities"][nameVar]=dataVar
                    next_id=consulta("select next_id from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0].split(",")
                    state=str(next_id[cont])
                    break
                cont=cont+1
            else:
                state=str(consulta("select default_id from bloqueqrdinamico where id_robot='"+mensaje["payload"][0]["robot"]+"' and namestate='"+estado_actual+"'")[0])
            cont=confirma(ch,props, method, state,mensaje)


channel.basic_qos(prefetch_count=1)
channel.basic_consume(on_request, queue='chatbotV12')

print(" [x] Comenzando conversación...")
channel.start_consuming()

 [x] Comenzando conversación...
Mensaje User=>  {'payload': [{'robot': 'AAA1', 'recipient_id': '1981139385336437', 'content': 'Hola', 'state': 'none', 'blocktype': 'none', 'opciones': 'none', 'entitites': ' ', 'entities': {}}]}
id_usuario->  1981139385336437
id_robot:  AAA1
 [x] Enviando...  {"Respuesta": [{"content": "Hola", "next_id": "InfoRobot", "blockType": "informativo", "contentType": "text", "typingTime": "1", "payload": [{"content": "", "state": "Saludo", "recipient_id": "1981139385336437", "entities": {}}]}]}
Mensaje User=>  {'payload': [{'robot': 'AAA1', 'recipient_id': '1981139385336437', 'content': 'hii', 'state': 'none', 'blocktype': 'none', 'opciones': 'none', 'entitites': ' ', 'entities': {}}]}
id_usuario->  1981139385336437
Siguiente estado... InfoRobot
 [x] Enviando...  {"Respuesta": [{"content": "Este es el robot para conusltar info de ECOBICI", "next_id": "OpMenu", "blockType": "informativo", "contentType": "text", "typingTime": "2", "payload": [{"content": "", "sta

KeyboardInterrupt: 